[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shang-vikas/series1-coding-exercises/blob/main/exercises/blog-08/exercise-01.ipynb)

# 🧪 Exercise 2 — Build Self-Attention From Scratch

## Goal

Make attention feel mechanical.

In [ ]:
import torch
import torch.nn.functional as F

batch = 1
seq = 4
dim = 8

x = torch.randn(batch, seq, dim)

Wq = torch.randn(dim, dim)
Wk = torch.randn(dim, dim)
Wv = torch.randn(dim, dim)

Q = x @ Wq
K = x @ Wk
V = x @ Wv

scores = Q @ K.transpose(-2, -1) / dim**0.5
weights = F.softmax(scores, dim=-1)

output = weights @ V

print("Attention matrix:")
print(weights[0])

Now explain:

Each row is a routing decision.

Each token decides how much every other token matters.

## Modify One Token

Change token 0 slightly.

In [ ]:
# Modify token 0
x_modified = x.clone()
x_modified[0, 0] += 0.5  # Small change to first token

Q_modified = x_modified @ Wq
K_modified = x_modified @ Wk
V_modified = x_modified @ Wv

scores_modified = Q_modified @ K_modified.transpose(-2, -1) / dim**0.5
weights_modified = F.softmax(scores_modified, dim=-1)

print("Original attention matrix:")
print(weights[0])
print("\nModified attention matrix:")
print(weights_modified[0])
print("\nDifference:")
print((weights - weights_modified)[0])

**Observe attention matrix changes everywhere.**

This demonstrates contextualization.